In [1]:
## Importing Libraries
import pandas as pd
import numpy as np

In [2]:
## Reading only the filtered dataset
df_filtered = pd.read_csv('df_filtered.csv')

In [3]:
df_filtered.head(1)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,is_purchase,price_category,treatment,post_treatment,interaction,price_category_num,interaction_impact,category_code_orig,brand_orig
0,2019-10-01,purchase,1004856,2053013555631882655,100,1585,130.76,1745658,8077710,1,Medium,1,0,0,1,0.005314,electronics.smartphone,samsung


In [4]:
# Convert 'event_time' to datetime 
df_filtered['event_time'] = pd.to_datetime(df_filtered['event_time'])

In [5]:
# Sort the DataFrame by event_time to ensure latest events are at the bottom
df_filtered = df_filtered.sort_values(by='event_time')

In [9]:
# Group by category and pick the last row (latest interaction impact) for each category
df_last_impact = df_filtered.groupby(['category_code', 'price_category_num']).last().reset_index()

In [11]:
df_last_impact.head(1)

,category_code,price_category_num,event_time,event_type,product_id,category_id,brand,price,user_id,user_session,is_purchase,price_category,treatment,post_treatment,interaction,interaction_impact,category_code_orig,brand_orig
0,0,0,2019-11-30,view,18300913,2053013558945383017,828,47.29,2610876,233565,0,Low,1,1,1,0.023722,accessories.bag,hp


In [12]:
# Keep only the relevant columns 
df_last_impact = df_last_impact[['category_code_orig','brand_orig', 'price_category','price_category_num', 'interaction_impact', 'event_time']]

In [13]:
# Display the resulting DataFrame
print(df_last_impact.head(1))

  category_code_orig brand_orig price_category  price_category_num  \
0    accessories.bag         hp            Low                   0   

   interaction_impact event_time  
0            0.023722 2019-11-30  


In [14]:
## Loading df

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
# Load the datasets
oct_df = pd.read_csv('/data/CausalTrial/2019-Oct.csv')  # Update the path to your October dataset
nov_df = pd.read_csv('/data/CausalTrial/2019-Nov.csv')  # Update the path to your November dataset


In [18]:
# Combine the data
df = pd.concat([oct_df, nov_df])

In [19]:
del(oct_df)
del(nov_df)

In [20]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [21]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [22]:
df.dropna(inplace=True)

In [23]:
df.isnull().sum()

event_time       0
event_type       0
product_id       0
category_id      0
category_code    0
brand            0
price            0
user_id          0
user_session     0
dtype: int64

In [24]:
# Step 1: Filter the dataset to include only purchase events
df['is_purchase'] = df['event_type'].apply(lambda x: 1 if x == 'purchase' else 0)

In [25]:
### Filtering the price into 4 categories
df['price_category'] = pd.qcut(df['price'], q=4, labels=['Low', 'Medium', 'High', 'VeryHigh'])


In [26]:
# Step 1: Group by user to see how many purchases were made in each price category
user_purchases = df[df['is_purchase'] == 1].groupby(['user_id', 'price_category']).agg({
    'product_id': 'count',  # Count of purchases
    'user_session': 'first'  # Keep the first user session (could also use mode or another aggregation method)
}).reset_index()

# Rename product_id to purchase_count for clarity
user_purchases.rename(columns={'product_id': 'purchase_count'}, inplace=True)

In [28]:
# Step 2: Determine the primary price category for each user
# We will create a function that will assign a primary category based on the majority of purchases

def assign_user_category(group):
    # If there's only one row for this user, return that price category
    if len(group) == 1:
        return group['price_category'].values[0]
   
    # If the user has multiple purchases, check which category has the most purchases
    # Sort by purchase count and take the one with the highest count
    max_purchase_count = group['purchase_count'].max()
    majority_categories = group[group['purchase_count'] == max_purchase_count]['price_category'].tolist()

    # If the user has purchased equally from multiple categories, return a list of those categories
    return majority_categories

# Apply the assign_user_category function to each user
user_primary_category = user_purchases.groupby('user_id').apply(assign_user_category).reset_index()

# Rename columns for clarity
user_primary_category.columns = ['user_id', 'user_category']

In [29]:
# Step 3: Add user_session back to the user DataFrame
# We'll merge it back using the user_purchases dataframe
user_primary_category = user_primary_category.merge(user_purchases[['user_id', 'user_session']].drop_duplicates(), on='user_id')

In [30]:
user_primary_category

,user_id,user_category,user_session
0,225644257,[Low],3601c672-9a75-4627-b9be-576298187981
1,225644257,[Low],NaN
2,253299396,[High],NaN
3,253299396,[High],e38b620b-6024-44e8-9db8-81a3a5b0e7e7
4,256164170,[Medium],NaN
...,...,...,...
1176227,579964521,[High],a185a06e-2d7c-4b26-b615-61aa755366ae
1176228,579966375,[Medium],NaN
1176229,579966375,[Medium],b9a6c805-1b4c-4c46-9a49-95dfbcf82750
1176230,579967058,[Medium],NaN


In [31]:
user_primary_category.user_category.value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[Medium]                         332862
[High]                           268036
[VeryHigh]                       236117
[Low]                            205527
[Medium, High]                    31859
[Low, Medium]                     27569
[High, VeryHigh]                  21548
[Medium, VeryHigh]                15793
[Low, High]                       15233
[Low, VeryHigh]                    8839
[Low, Medium, High]                4428
[Medium, High, VeryHigh]           3930
[Low, Medium, VeryHigh]            2072
[Low, High, VeryHigh]              1765
[Low, Medium, High, VeryHigh]       654
Name: user_category, dtype: int64

In [32]:
user_primary_category_exp = user_primary_category.explode('user_category')
user_primary_category_exp

,user_id,user_category,user_session
0,225644257,Low,3601c672-9a75-4627-b9be-576298187981
1,225644257,Low,NaN
2,253299396,High,NaN
3,253299396,High,e38b620b-6024-44e8-9db8-81a3a5b0e7e7
4,256164170,Medium,NaN
...,...,...,...
1176227,579964521,High,a185a06e-2d7c-4b26-b615-61aa755366ae
1176228,579966375,Medium,NaN
1176229,579966375,Medium,b9a6c805-1b4c-4c46-9a49-95dfbcf82750
1176230,579967058,Medium,NaN


In [33]:
## Now mapping the category with the list of users who will be interested

In [34]:
# Step 1: Merge user_primary_category_expanded with df_last_impact based on price_category
# Here, user_category in user_primary_category_expanded is the same as price_category_num in df_last_impact
df_merged = pd.merge(user_primary_category_exp,
                     df_last_impact,
                     left_on='user_category',
                     right_on='price_category',
                     how='inner')

In [37]:
df_merged.shape

(75770814, 9)

In [41]:
# Step 2: Group by price_category_num (or user_category) and list users
grouped_users = df_merged.groupby('price_category').agg({
    'user_id': list,           # Aggregate the user IDs into a list
    'user_session': list,      # Aggregate the user sessions into a list
    'interaction_impact': 'median'  # keeping the interaction impact with median impact since different category has differnt products imapct reference
}).reset_index()

In [42]:
grouped_users

,price_category,user_id,user_session,interaction_impact
0,High,"[253299396, 253299396, 253299396, 253299396, 2...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0.035695
1,Low,"[225644257, 225644257, 225644257, 225644257, 2...","[3601c672-9a75-4627-b9be-576298187981, 3601c67...",0.037824
2,Medium,"[256164170, 256164170, 256164170, 256164170, 2...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0.034785
3,VeryHigh,"[304707635, 304707635, 304707635, 304707635, 3...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0.033145


In [43]:
grouped_users.to_csv('userList_persuadables_DiD.csv', index=None)

- Taken only the filtered dataframe with the details of the product brand which has positive impact
- Now we need to find which user segment will be influenced by this price change
- Based on the grouping we have found out the group of users who will be influenced by the price change
- The userList_persuadables_DiD.csv has the list of users who can be persuaded for the product purchase if the price is reduced.